# Infinite Families for the Godsil-Lato Graph on 8 Vertices

Is there an infinite family that generalizes the Godsil-Lato graph on 8 vertices, whose adjacency matrix is $I_2 \otimes H(C_4) + \sigma_y \otimes J_4$?

In [1]:
# %load adj_functions.sage
import numpy as np
from numpy import linalg

################################################################################
# Function: idempotents                                                        #
#                                                                              #
# Purpose: find principal idempotents of a Hermitian matrix                    #
#                                                                              #
# Arguments:                                                                   #
#   A        Hermitian sage or numpy matrix     adjacency matrix/Hamiltonian   #
#                                                                              #
# Returns: principal idempotents, eigenvalues                                  #
#                                                                              #
################################################################################

def idempotents(A):
    if not matrix(A).is_hermitian():
        print("A is not Hermitian")
        return []
    # Extracts eigenvalues and eigenvectors
    W,V = linalg.eigh(A)
    # Converts eigenvectors into orthonormal basis
    Q, R = linalg.qr(V) 
    Q = matrix(Q)
    # Number of eigenvectors
    A = np.matrix(A)
    n = len(A)
    idems = []
    evals = []
    i = 0
    j = 0
    while i < n:
        # Zero matrix
        E = matrix(n, n, 0)
        # Checks if eigenvectors share an eigenvalue
        while j < n and abs(W[i] - W[j]) < 0.01:
            # Converts eigenvector into column vecror
            v = Q.column(j).column()
            # updates idempotent
            E = E + v*v.conjugate_transpose();
            j = j + 1
        E = E
        idems.append(E)
        evals.append(W[i].round(3))
        i = j
    return idems, evals

################################################################################
# Function: transfer                                                           #
#                                                                              #
# Purpose: return transfer matrix U(t) given adjacency matrix/Hamiltonian      #
#                                                                              #
# Arguments:                                                                   #
#   A        Hermitian sage or numpy matrix     adjacency matrix/Hamiltonian   #
#   t        float                              time                           #
#   k        integer                            rounding accuracy              #
#                                                                              #
# Returns: transfer U(t)                                                       #
#                                                                              #
################################################################################

def transfer(A, t):
    if not matrix(A).is_hermitian():
        print("A is not Hermitian")
        return []
    idems, evals = idempotents(A)
    # Number of eigenvectors
    A = np.matrix(A)
    n = len(A)
    # Zero matrix
    U = matrix(n, n, 0)
    for i in range(len(idems)):
        # Updates transder matrix
        U = U + exp(-I*t*evals[i])*idems[i]
    return U
#     A = matrix(A)
#     return matrix(exp(-1*I*t*A))

################################################################################
# Function: transfer_at_time_r                                                 #
#                                                                              #
# Purpose: find real part of transfer matrix at specific time and index        #
#                                                                              #
# Arguments:                                                                   #
#   A        Hermitian sage or numpy matrix     adjacency matrix/Hamiltonian   #
#   i        integer                            row number                     #
#   j        integer                            column number                  #
#                                                                              #
# Returns: real part of U(t) at index [i][j]                                   #
#                                                                              #
################################################################################

def transfer_at_time_r(A,t,i,j):
    U = transfer(A,t)
    if abs(U[i][j].real()) < 0.00001:
        return 0
    return U[i][j].real()

################################################################################
# Function: transfer_at_time_i                                                 #
#                                                                              #
# Purpose: find imaginary part of transfer matrix at specific time and index   #
#                                                                              #
# Arguments:                                                                   #
#   A        Hermitian sage or numpy matrix     adjacency matrix/Hamiltonian   #
#   i        integer                            row number                     #
#   j        integer                            column number                  #
#                                                                              #
# Returns: imaginary part of U(t) at index [i][j]                              #
#                                                                              #
################################################################################

def transfer_at_time_i(A,t,i,j):
    U = transfer(A,t)
    if abs(U[i][j].imag()) < 0.00001:
        return 0
    return U[i][j].imag()

################################################################################
# Function: transfer_at_time_a                                                 #
#                                                                              #
# Purpose: find absolute value of transfer matrix at specific time and index   #
#                                                                              #
# Arguments:                                                                   #
#   A        Hermitian sage or numpy matrix     adjacency matrix/Hamiltonian   #
#   i        integer                            row number                     #
#   j        integer                            column number                  #
#                                                                              #
# Returns: absolute value of U(t) at index [i][j]                              #
#                                                                              #
################################################################################

def transfer_at_time_a(A,t,i,j):
    U = transfer(A,t)
    if U[i][j].abs() < 0.00001:
        return 0
    return U[i][j].abs()
    
################################################################################
# Function: parallel                                                           #
#                                                                              #
# Purpose: check whether two vertices are parallel                             #
#                                                                              #
# Arguments:                                                                   #
#   idems    list of matrices     idempotents of an adjacency matrix           #
#   a        integer              first vertex                                 #
#   b        integer              second vertex                                #
#                                                                              #
# Returns: boolean whether vertices are parallel                               #
#                                                                              #
################################################################################

def parallel(a,b,idems):
    for M in idems:
        va = M[a].conjugate()
        vb = M[b]
        anorm=np.sqrt(M[a].dot_product(va))
        bnorm=np.sqrt(M[b].dot_product(M[b].conjugate()))
        if abs(abs(va.dot_product(vb))-anorm*bnorm)>0.0001:
            return False
    return True
    
################################################################################
# Function: cospec                                                             #
#                                                                              #
# Purpose: check whether two vertices are parallel                             #
#                                                                              #
# Arguments:                                                                   #
#   idems    list of matrices     idempotents of an adjacency matrix           #
#   a        integer              first vertex                                 #
#   b        integer              second vertex                                #
#                                                                              #
# Returns: boolean whether vertices are cospectral                             #
#                                                                              #
################################################################################

def cospec(a,b,idems):
    for M in idems:
        if abs(M[a,a]- M[b,b])>0.001:
            return False
    return True

################################################################################
# Function: strcospec                                                          #
#                                                                              #
# Purpose: check whether two vertices are parallel                             #
#                                                                              #
# Arguments:                                                                   #
#   idems    list of matrices     idempotents of an adjacency matrix           #
#   a        integer              first vertex                                 #
#   b        integer              second vertex                                #
#                                                                              #
# Returns: boolean whether vertices are strongly cospectral                    #
#                                                                              #
################################################################################

def strcospec(a,b,idems):
    if cospec(a,b,idems) and parallel(a,b,idems):
        return True
    return False
    
################################################################################
# Function: transfer_plots                                                     #
#                                                                              #
# Purpose: list plots for each entry of transfer matrix from 0 to 3pi          #
#                                                                              #
# Arguments:                                                                   #
#   A        Hermitian sage or numpy matrix     adjacency matrix/Hamiltonian   #
#   s        character                          choose 'r','i', or 'a'         #
#                                               for real, imaginary, or abs    #
#                                                                              #
# Returns: absolute value of U(t) at index [i][j]                              #
#                                                                              #
################################################################################

def transfer_plots(A,s):
    A = np.matrix(A)
    n = len(A)
    plots = []
    if s == 'r':
        for i in range(n):
            for j in range(n):
                plots.append(plot(transfer_at_time_r(A,x,i,j),0,3*np.pi))
    elif s == 'i':
        for i in range(n):
            for j in range(n):
                plots.append(plot(transfer_at_time_i(A,x,i,j),0,3*np.pi))
    else:
        for i in range(n):
            for j in range(n):
                plots.append(plot(transfer_at_time_a(A,x,i,j),0,3*np.pi))
    chunks = [plots[x:x+n] for x in range(0, len(plots), n)]
    return chunks

# Base case

In [2]:
IB = identity_matrix(2)
HC4B = np.matrix([[0,-1j,0,1j],[1j,0,-1j,0],[0,1j,0,-1j],[-1j,0,1j,0]]); HC4B = matrix(HC4B)
SYB = np.matrix([[0,-1j],[1j,0]]); SYB = matrix(SYB)
J4B = ones_matrix(4,4)
AB =  IB.tensor_product(HC4B) + SYB.tensor_product(J4B); AB

[   0.0 -1.0*I    0.0  1.0*I -1.0*I -1.0*I -1.0*I -1.0*I]
[ 1.0*I    0.0 -1.0*I    0.0 -1.0*I -1.0*I -1.0*I -1.0*I]
[   0.0  1.0*I    0.0 -1.0*I -1.0*I -1.0*I -1.0*I -1.0*I]
[-1.0*I    0.0  1.0*I    0.0 -1.0*I -1.0*I -1.0*I -1.0*I]
[ 1.0*I  1.0*I  1.0*I  1.0*I    0.0 -1.0*I    0.0  1.0*I]
[ 1.0*I  1.0*I  1.0*I  1.0*I  1.0*I    0.0 -1.0*I    0.0]
[ 1.0*I  1.0*I  1.0*I  1.0*I    0.0  1.0*I    0.0 -1.0*I]
[ 1.0*I  1.0*I  1.0*I  1.0*I -1.0*I    0.0  1.0*I    0.0]

In [3]:
transfer(AB, np.pi/4).round(5)

[ 0.0 -1.0  0.0  0.0  0.0  0.0  0.0  0.0]
[ 0.0  0.0 -1.0  0.0  0.0  0.0  0.0  0.0]
[ 0.0  0.0  0.0 -1.0  0.0  0.0  0.0  0.0]
[-1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0]
[ 0.0  0.0  0.0  0.0  0.0 -1.0  0.0  0.0]
[ 0.0  0.0  0.0  0.0  0.0  0.0 -1.0  0.0]
[ 0.0  0.0  0.0  0.0  0.0  0.0  0.0 -1.0]
[ 0.0  0.0  0.0  0.0 -1.0  0.0  0.0  0.0]

In [4]:
E, eVal = idempotents(AB)
for i in range(len(eVal)):
    print("Idempotent for", int(eVal[i]))
    print(E[i].round(5))

Idempotent for -4
[   0.125    0.125    0.125    0.125  0.125*I  0.125*I  0.125*I  0.125*I]
[   0.125    0.125    0.125    0.125  0.125*I  0.125*I  0.125*I  0.125*I]
[   0.125    0.125    0.125    0.125  0.125*I  0.125*I  0.125*I  0.125*I]
[   0.125    0.125    0.125    0.125  0.125*I  0.125*I  0.125*I  0.125*I]
[-0.125*I -0.125*I -0.125*I -0.125*I    0.125    0.125    0.125    0.125]
[-0.125*I -0.125*I -0.125*I -0.125*I    0.125    0.125    0.125    0.125]
[-0.125*I -0.125*I -0.125*I -0.125*I    0.125    0.125    0.125    0.125]
[-0.125*I -0.125*I -0.125*I -0.125*I    0.125    0.125    0.125    0.125]
Idempotent for -2
[   0.25  0.25*I   -0.25 -0.25*I     0.0     0.0     0.0     0.0]
[-0.25*I    0.25  0.25*I   -0.25     0.0     0.0     0.0     0.0]
[  -0.25 -0.25*I    0.25  0.25*I     0.0     0.0     0.0     0.0]
[ 0.25*I   -0.25 -0.25*I    0.25     0.0     0.0     0.0     0.0]
[    0.0     0.0     0.0     0.0    0.25  0.25*I   -0.25 -0.25*I]
[    0.0     0.0     0.0     0.0 -0.25*I  

# Case which gives MST?

Adjacency matrix is $$I_{2^n} \otimes H(C_4) + \underbrace{\sigma_Y \otimes \cdots \otimes \sigma_Y}_{n \text { times }}  \otimes J_4$$ for some even positive integer $n$.

In [5]:
def GL_PST_EX2(n):
    HC4 = np.matrix([[0,-1j,0,1j],[1j,0,-1j,0],[0,1j,0,-1j],[-1j,0,1j,0]]); HC4 = matrix(HC4)
    SY = np.matrix([[0,-1j],[1j,0]]); SY = matrix(SY)
    Id = identity_matrix(2^n)
    J4 = ones_matrix(4,4)
    A = Id.tensor_product(HC4)
    SYs = SY
    for k in range(n-1):
        SYs = SYs.tensor_product(SY)
    B = SYs.tensor_product(J4)
    print(A*B - B*A == 0)
    return A + B

In [6]:
n = 2
X = GL_PST_EX2(n)
print("t = pi/4")
UX1 = transfer(X, np.pi/4).round(5)
for k in range(2^n*4):
    print([round(x.real(),0) for x in UX1[k]])
print("t = pi/2")
UX2 = transfer(X, np.pi/2).round(5)
for k in range(2^n*4):
    print([round(x.real(),0) for x in UX2[k]])
print("t = 3pi/4")
UX3 = transfer(X, 3*np.pi/4).round(5)
for k in range(2^n*4):
    print([round(x.real(),0) for x in UX3[k]])
print("t = pi")
UX4 = transfer(X, np.pi).round(5)
for k in range(2^n*4):
    print([round(x.real(),0) for x in UX4[k]])

True
t = pi/4
[0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0]
t = pi/2
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0,

In [7]:
EX, eValX = idempotents(X)
for val in eValX:
    print(val.round(5))

-4.0
-2.0
-0.0
2.0
4.0


# Case which gives pairs of PST but not MST

Adjacency matrix is $I_n \otimes H(C_4) + \sigma_y \otimes J_{2n}$ for some even positive integer $n$.

In [8]:
def GL_PST_EX(n):
    HC4 = np.matrix([[0,-1j,0,1j],[1j,0,-1j,0],[0,1j,0,-1j],[-1j,0,1j,0]]); HC4 = matrix(HC4)
    SY = np.matrix([[0,-1j],[1j,0]]); SY = matrix(SY)
    Id = identity_matrix(n)
    J = ones_matrix(2*n,2*n)
    A = Id.tensor_product(HC4)
    B = SY.tensor_product(J)
    return A + B

In [9]:
m = 4
Y = GL_PST_EX(m)
UY = transfer(Y, np.pi/2).round(5)
UY

[0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0]
[0.0 0.0 0

In [10]:
EY, eValY = idempotents(Y)
for val in eValY:
    print(val.round(5))

-8.0
-2.0
-0.0
2.0
8.0
